<p>
Homework: https://work.caltech.edu/homework/hw7.pdf
<p>

&#x2714;
Answers:
1. d &#x2714;
2. e &#x2714;
3. d &#x2714;
4. d &#x2714;
5. b &#x2714;
6. d &#x2714;
7. 
8. 
9. 
10. 

<p>
Answer key: https://work.caltech.edu/homework/hw7_sol.pdf

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

In [2]:
# Questions 1 through 5
# training set: http://work.caltech.edu/data/in.dta
# test set: http://work.caltech.edu/data/out.dta
train = pd.read_fwf('http://work.caltech.edu/data/in.dta', header=None)
train_x = train.iloc[:, :2]
train_y = train.iloc[:, 2]

test = pd.read_fwf('http://work.caltech.edu/data/out.dta', header=None)
test_x = test.iloc[:, :2]
test_y = test.iloc[:, 2]

In [3]:
def transform(x):
    x1 = x.iloc[:, 0]
    x2 = x.iloc[:, 1]
    
    z = pd.DataFrame(np.ones(len(x)))
    z.loc[:, 1] = x1
    z.loc[:, 2] = x2
    z.loc[:, 3] = x1 ** 2
    z.loc[:, 4] = x2 ** 2
    z.loc[:, 5] = x1 * x2
    z.loc[:, 6] = np.abs(x1 - x2)
    z.loc[:, 7] = np.abs(x1 + x2)
    return z

In [4]:
train_z = transform(train_x)
test_z = transform(test_x)

In [5]:
from numpy.linalg import inv
def linear_regression(Z, y):
    Z_T_Z = np.dot(Z.T, Z)
    Z_T_Z_inverse = inv(Z_T_Z)
    Z_dagger = np.dot(Z_T_Z_inverse, Z.T)
    weights = np.dot(Z_dagger, y)
    return weights

In [6]:
def error(W, X, y):
    misclassified = np.sign(np.dot(X, W)) != y
    return 1.0 * misclassified.sum()/len(misclassified)

In [22]:
def calc_error(X, y, X_val, y_val, X_test, y_test):
    weights = linear_regression(X, y)
    print "train   ", np.round(error(weights, X, y), 8)
    print "validate", np.round(error(weights, X_val, y_val), 8)
    print "test    ", np.round(error(weights, X_test, y_test), 8)

In [23]:
train_set_size = 25

z_train = train_z.head(train_set_size)
y_train = train_y.head(train_set_size)

z_validate = train_z.tail(len(train_z) - train_set_size)
y_validate = train_y.tail(len(train_z) - train_set_size)

z_test = test_z
y_test = test_y

In [38]:
for k in [3, 4, 5, 6, 7]:
    print "Trying k: ", k
    calc_error(z_train.ix[:,0:k], y_train, 
               z_validate.ix[:,0:k], y_validate,
               z_test.ix[:,0:k], y_test)

Trying k:  3
train    0.44
validate 0.3
test     0.42
Trying k:  4
train    0.32
validate 0.5
test     0.416
Trying k:  5
train    0.08
validate 0.2
test     0.188
Trying k:  6
train    0.04
validate 0.0
test     0.084
Trying k:  7
train    0.04
validate 0.1
test     0.072


In [39]:
for k in [3, 4, 5, 6, 7]:
    print "Trying k: ", k
    calc_error(z_validate.ix[:,0:k], y_validate,
               z_train.ix[:,0:k], y_train, 
               z_test.ix[:,0:k], y_test)

Trying k:  3
train    0.4
validate 0.28
test     0.396
Trying k:  4
train    0.3
validate 0.36
test     0.388
Trying k:  5
train    0.2
validate 0.2
test     0.284
Trying k:  6
train    0.0
validate 0.08
test     0.192
Trying k:  7
train    0.0
validate 0.12
test     0.196


In [41]:
#Question 6

N=1000
e1 = np.random.uniform(size=N)
e2 = np.random.uniform(size=N)
e = np.minimum(e1, e2)

print np.mean(e1), np.mean(e2), np.mean(e)

0.502402067838 0.501660183482 0.336991151062
